In [5]:
from SeoKeywordResearch import SeoKeywordResearch
from dotenv import load_dotenv
import os


from pathlib import Path




In [7]:
env_path = Path('.') / 'api.env'
load_dotenv(dotenv_path=env_path)

True

In [9]:
import time
import json
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from SeoKeywordResearch import SeoKeywordResearch



load_dotenv(dotenv_path=env_path)

SERPAPI_KEY = os.getenv("SERPAPI_KEY")



In [11]:
def init_chrome_driver():
    """Initialize and return a Chrome webdriver with appropriate options."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver



In [13]:
def scrape_amazon_best_sellers(category_url, num_products=10):
    """
    Scrape Amazon Best Sellers for a specific category
    
    Args:
        category_url: URL of the Amazon Best Sellers category
        num_products: Number of products to scrape (default: 10)
        
    Returns:
        DataFrame containing product information
    """
    driver = init_chrome_driver()
    
    try:
        print(f"Accessing {category_url}...")
        driver.get(category_url)
        time.sleep(3)  
    
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
 
        product_items = soup.select("div[id^='gridItemRoot']")
        print(f"Found {len(product_items)} products")
        
        products = []
        count = 0
        
        for item in product_items:
            if count >= num_products:
                break
                
            try:
               
                product_data = {}
                
          
                rank_elem = item.select_one("span.zg-bdg-text")
                if rank_elem:
                    product_data["rank"] = rank_elem.text.strip().replace("#", "")
                
                # title
                title_elem = item.select_one("div._cDEzb_p13n-sc-css-line-clamp-3_g3dy1")
                if title_elem:
                    product_data["title"] = title_elem.text.strip()
                
                #  price
                price_elem = item.select_one("span.p13n-sc-price")
                if price_elem:
                    product_data["price"] = price_elem.text.strip()
                
                #rating
                rating_elem = item.select_one("span.a-icon-alt")
                if rating_elem:
                    product_data["rating"] = rating_elem.text.split(" out of")[0]
                
                # URL
                link_elem = item.select_one("a.a-link-normal")
                if link_elem and "href" in link_elem.attrs:
                    product_data["url"] = "https://www.amazon.com" + link_elem["href"]
                
             
                img_elem = item.select_one("img.a-dynamic-image")
                if img_elem and "src" in img_elem.attrs:
                    product_data["image_url"] = img_elem["src"]
                
           
                if "title" in product_data and "url" in product_data:
                    products.append(product_data)
                    count += 1
                    print(f"Scraped product {count}: {product_data['title'][:40]}...")
            
            except Exception as e:
                print(f"Error scraping product: {e}")
                continue
        
        return pd.DataFrame(products)
    
    finally:
        driver.quit()



In [15]:
def get_seo_keywords(product_title, num_keywords=4):
    """
    Use SerpApi to get relevant SEO keywords for a product with improved handling
    """
    try:
    
        short_title = ' '.join(product_title.split()[:3])
        
        keyword_research = SeoKeywordResearch(
            query=short_title,  # Use shorter title
            api_key=os.getenv("seokeywordapikey"),
          
            lang='en',
            country='us',
            domain='google.com'
        )
        
       
        auto_complete_results = keyword_research.get_auto_complete()
        related_searches_results = keyword_research.get_related_searches()
        related_questions_results = keyword_research.get_related_questions(depth_limit=1)  # Increase depth
        
        #now we will combine all the results  that we habe got till now
        all_keywords = []
        if auto_complete_results:
            all_keywords.extend(auto_complete_results)
        if related_searches_results:
            all_keywords.extend(related_searches_results)
        if related_questions_results:
            all_keywords.extend(related_questions_results)
        
        # if keywords found - return them .....
        if all_keywords:
            return all_keywords[:num_keywords]
            
        # If no keywords found
        if len(short_title.split()) > 1:
            keyword_research = SeoKeywordResearch(
                query=short_title.split()[0],  # Just the first word
                api_key=os.getenv("seokeywordapikey"),
                lang='en',
                country='us',
                domain='google.com'
            )
            
            auto_complete_results = keyword_research.get_auto_complete()
            if auto_complete_results:
                return auto_complete_results[:num_keywords]
        
        # Fallback keywords
        if "fire tv" in product_title.lower() or "firestick" in product_title.lower():
            return ["fire tv stick setup", "fire stick vs roku", "best fire tv apps", "fire tv 4k vs hd", "fire tv stick troubleshooting"]
            
        # Default fallback
        return [short_title]
        
    except Exception as e:
        print(f"Error getting keywords for '{product_title}': {e}")
        # Fallback for streaming devices
        if "fire tv" in product_title.lower() or "firestick" in product_title.lower():
            return ["fire tv stick setup", "fire stick vs roku", "best fire tv apps", "fire tv 4k vs hd", "fire tv stick troubleshooting"]
        return [product_title.split()[0]]



In [17]:
def main():
    # now defining the categories
    categories = {
        "Kitchen & Dining": "https://www.amazon.com/Best-Sellers-Home-Kitchen-Kitchen-Dining/zgbs/home-garden/284507/",
        "Electronics": "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/",
        "Clothing": "https://www.amazon.com/Best-Sellers-Clothing-Shoes-Jewelry/zgbs/fashion/",
        "Beauty & Personal Care": "https://www.amazon.com/Best-Sellers-Beauty-Personal-Care/zgbs/beauty/",
        "Toys & Games": "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/"
    }
    
  
    print("Available categories:")
    for i, category in enumerate(categories.keys(), 1):
        print(f"{i}. {category}")

    while True:
        try:
            category_choice = int(input("\nSelect a category (enter number): "))
            if 1 <= category_choice <= len(categories):
                break
            else:
                print(f"Please enter a number between 1 and {len(categories)}")
        except ValueError:
            print("Please enter a valid number")
    
    category_name = list(categories.keys())[category_choice-1]
    category_url = categories[category_name]
    

    print(f"\nScraping products from {category_name} category...")
    products_df = scrape_amazon_best_sellers(category_url, num_products=5)
    
    # now we will only display the top 5 products to the user so it is easier for him to select any one out of those
    print("\nTop 5 Products:")
    for i, row in products_df.iterrows():
        price = row.get('price', 'N/A')
        rating = row.get('rating', 'N/A')
        print(f"{i+1}. {row['title']} - Price: {price} - Rating: {rating}")
    
    # get user input for product selection
    while True:
        try:
            product_choice = int(input("\nSelect a product for SEO blog creation (enter number): "))
            if 1 <= product_choice <= len(products_df):
                break
            else:
                print(f"Please enter a number between 1 and {len(products_df)}")
        except ValueError:
            print("Please enter a valid number")
    
    selected_product = products_df.iloc[product_choice-1]
    
    print(f"\nYou selected: {selected_product['title']}")
    
    # get SEO keywords for the selected product
    print("\nResearching SEO keywords...")
    keywords = get_seo_keywords(selected_product['title'])
    
    print(f"\nSEO Keywords for '{selected_product['title']}': {keywords}")
    
    # Store the selected product and keywords for the next phase (blog generation)
    selected_data = {
        'product': selected_product.to_dict(),
        'keywords': keywords
    }
    

    with open('selected_product_data.json', 'w') as f:
        json.dump(selected_data, f)
    
    print("\nProduct and keyword data saved for blog generation phase.")

if __name__ == "__main__":
    main()



Available categories:
1. Kitchen & Dining
2. Electronics
3. Clothing
4. Beauty & Personal Care
5. Toys & Games



Select a category (enter number):  3



Scraping products from Clothing category...
Accessing https://www.amazon.com/Best-Sellers-Clothing-Shoes-Jewelry/zgbs/fashion/...
Found 30 products
Scraped product 1: Crocs Unisex Adult Classic Clog...
Scraped product 2: Gildan Platinum Men's Crew T-Shirts...
Scraped product 3: ANRABESS Womens Short Sleeve Henley Tops...
Scraped product 4: ATHMILE Womens Oversized T Shirts Loose ...
Scraped product 5: Nippies Nipple Covers for Women - Sticky...

Top 5 Products:
1. Crocs Unisex Adult Classic Clog - Price: nan - Rating: 4.8
2. Gildan Platinum Men's Crew T-Shirts - Price: nan - Rating: 4.6
3. ANRABESS Womens Short Sleeve Henley Tops V Neck Dressy Casual Loose Fit Shirts Summer Flowy Blouses 2025 Trendy Clothes - Price: $9.99 - Rating: 4.3
4. ATHMILE Womens Oversized T Shirts Loose Fit Crewneck Short Sleeve Tops Summer Casual Blouse Y2K 2025 Basic Tee Beach - Price: $7.99 - Rating: 4.4
5. Nippies Nipple Covers for Women - Sticky Adhesive Silicone Pasties - Reusable Pasty Nipple Cover with


Select a product for SEO blog creation (enter number):  1



You selected: Crocs Unisex Adult Classic Clog

Researching SEO keywords...

SEO Keywords for 'Crocs Unisex Adult Classic Clog': ['crocs unisex adult classic clog', 'crocs unisex adult echo clog', 'crocs unisex adult crocband clog', 'crocs unisex adult classic lined clog']

Product and keyword data saved for blog generation phase.


In [19]:
import json
import os
import requests
from dotenv import load_dotenv
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow




PERPLEXITY_API_KEY = os.getenv("perplexityapi")

SCOPES = ['https://www.googleapis.com/auth/blogger']




In [21]:
def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            with open('cred.json') as f:
                cred_data = json.load(f)
            flow = InstalledAppFlow.from_client_config(cred_data, SCOPES)
            creds = flow.run_local_server(port=8080)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

def get_blog_id(creds):
    url = "https://www.googleapis.com/blogger/v3/users/self/blogs"
    headers = {"Authorization": f"Bearer {creds.token}"}
    response = requests.get(url, headers=headers)
    blogs = response.json().get('items', [])
    if not blogs:
        raise Exception("No blogs found on this account.")
    return blogs[0]['id']  # Automatically select the first blog



In [23]:
def create_post(creds, blog_id, title, content):
    url = f"https://www.googleapis.com/blogger/v3/blogs/{blog_id}/posts/"
    headers = {
        "Authorization": f"Bearer {creds.token}",
        "Content-Type": "application/json"
    }
    data = {
        "kind": "blogger#post",
        "blog": {"id": blog_id},
        "title": title,
        "content": content
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        post = response.json()
        print("✅ Post published!")
        print("📝 Title:", post['title'])
        print("🔗 URL:", post['url'])
    else:
        print("❌ Failed to publish post:", response.status_code, response.text)



In [25]:
def generate_blog_post(product_data, keywords, target_length=200):
    product_title = product_data.get('title', '')
    product_price = product_data.get('price', 'N/A')
    product_rating = product_data.get('rating', 'N/A')
    keywords_str = ', '.join(keywords)

    prompt = f"""Write a 150-200 word blog post about the product: {product_title}. 
Price: {product_price}
Rating: {product_rating}

Include these SEO keywords: {keywords_str}
Use an engaging title, highlight features, and conclude with a recommendation. Write in a friendly, informative tone."""

    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": "You are a professional blog writer specializing in SEO-optimized product reviews."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 500
    }

    response = requests.post("https://api.perplexity.ai/chat/completions", headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        content = result.get('choices', [{}])[0].get('message', {}).get('content', '').strip()
        return content
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")



In [27]:
def main():
    try:
        
        with open('selected_product_data.json', 'r') as f:
            data = json.load(f)
        product = data.get('product', {})
        keywords = data.get('keywords', [])

        print(f"🔄 Generating blog post for: {product.get('title', '')}")
        post_content = generate_blog_post(product, keywords)

        # auth and publish it finally 
        creds = authenticate()
        blog_id = get_blog_id(creds)
        create_post(creds, blog_id, product.get('title', 'Untitled'), post_content)

    except FileNotFoundError:
        print("❌ 'selected_product_data.json' not found.")
    except Exception as e:
        print(f"❌ Error: {e}")



In [29]:
if __name__ == "__main__":
    main()

🔄 Generating blog post for: Crocs Unisex Adult Classic Clog
✅ Post published!
📝 Title: Crocs Unisex Adult Classic Clog
🔗 URL: http://seoblogtask.blogspot.com/2025/05/crocs-unisex-adult-classic-clog.html
